# Data Clean up

Haytham Mohamed - INFS890 Spring 2020

This notebook is to check missing data and outliers

In [288]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
from sklearn import preprocessing
from datetime import datetime
from scipy import stats

sns.set()
sns.set(color_codes=True)
#sns.set_color_codes()

pd.options.display.max_rows = 15
pd.options.display.float_format = '{:,.2f}'.format

# Read the raw data. 

All data is in one file, where each metric is listed under each other with three columns: 

1- metric: The name of the metric

2- date: the data and time of the metric

3- value: the metric value


In [289]:
# 1- read processed file

home_dir = '/Users/hmohamed/github/data-research-spring2020/sock-shop'

file_dir = home_dir + '/metrics-1pod/'
processed_dir = home_dir + '/processed-data/' 


orders_ltcy_file = '1_orders_ltcy.csv'
carts_ltcy_file = '2_carts_ltcy.csv'
 
carts_req_rate_file = '3_carts_req.csv'
orders_req_rate_file = '4_orders_req.csv'

service_cpu_usage_file = '6_svc_cpu_use.csv'
service_cpu_sat_file = '8_svc_cpu_sat.csv'

nodes_cpu_usage_file = '7_nodes_cpu_use.csv'
nodes_cpu_sat_file = '9_nodes_cpu_sat.csv'

services_pods_file = '5_svc_pods.csv'


service_input_files = [service_cpu_usage_file
                      #, service_cpu_sat_file
                      #, system_cpu_usage_file
                      #, system_cpu_sat_file
                       
                      #, orders_ltcy_file
                      #, carts_ltcy_file
                     ]

concated_data_file = 'all_data.csv'

save=True
frequency = '15S'  # S for second , T for minute

In [290]:
def read_df(file_dir, data_file):
    df = pd.read_csv(file_dir + data_file, header=None, skiprows=1, names=['unit', 'metric', 'date', 'value'])
    #df = df.drop(['pods'], axis=1)
    return to_time_series(df)

def to_time_series(df, index_col_name='date', value_col_name='value'):
    df[index_col_name] = pd.to_datetime(df[index_col_name])
    df[value_col_name] = pd.to_numeric(df[value_col_name])
    df.set_index(index_col_name, inplace=True)
    df.sort_index(inplace=True)
    return df
    

# This function merges and alines the metrics timeseries data into a data frame, a column for every feature
def expand(df, by_col='unit', base=37, dup='mean'):
    # first convert to time series
    #df = toTimeSeries(df, 'date')
    by_col_values = np.unique(df[by_col])
    metrics_df = pd.DataFrame()
    i = 0
    for col_value in by_col_values:
        #print("Processing metric for column: %", col_value)
        series = extract_metric_series(df, by_col, col_value)
        series = resample(series, value_column_name=col_value, dup=dup, base=base)  
        if i == 0:
            metrics_df = series
        else:
            metrics_df = merge(metrics_df, series)
        i = i + 1
    return metrics_df 

# T for minutes, S for seconds
# remedy duplicates by either taking the maximum (=max) or average  (=mean) them
def resample(df, value_column_name, index_col_name='date', frequency = frequency, interpolate = True
             , interpolate_method = 'linear', base=37, dup = 'mean'):
    # eliminate dups in timestamp
    if dup == 'max':
        df = df.groupby([index_col_name])[value_column_name].max()   # taking max
        df = pd.DataFrame(df)
    elif dup == 'sum':
        df = df.groupby([index_col_name])[value_column_name].sum()
    else:
        df = df.groupby([index_col_name]).mean()   # taking mean
        
    df.index = pd.to_datetime(df.index)
    df.sort_index(inplace=True)  # order the timeseries
    
    # fill in missing interval (upsample)
    shape_before = df.shape
    resampled = df.resample(frequency, kind='timestamp', base=base).bfill()
    
    if interpolate:
        resampled = resampled.interpolate(method=interpolate_method)    
        
    #print("dimention before resampling is: {}".format(shape_before))
    return resampled


# This function extracts timeseries of one named service from the whole raw timeseries data
def extract_metric_series(df, col_name, col_value):
    metric = df.loc[df[col_name] == col_value].drop([col_name], axis=1).rename(index=str, columns={"value": col_value})
    metric.sort_index(inplace=True)
    return metric  

# sum df rows, remove expanded columns and set a new column with a metric name
def sum_timeseries(df, columns_to_delete, metric_name, metric_col_name='metric'):
    df['value'] = df.sum(axis=1)
    df[metric_col_name] = metric_name
    for col in columns_to_delete:
        df = df.drop([col], axis=1)
    return df 

# max df rows, remove expanded columns and set a new column with a metric name
def max_timeseries(df, columns_to_delete, metric_name, metric_col_name='metric'):
    df['value'] = df.max(axis=1)
    df[metric_col_name] = metric_name
    for col in columns_to_delete:
        df = df.drop([col], axis=1)
    return df

# average df rows, remove expanded columns and set a new column with a metric name
def avg_timeseries(df, columns_to_delete, metric_name, metric_col_name='metric'):
    df['value'] = df.mean(axis=1)
    df[metric_col_name] = metric_name
    for col in columns_to_delete:
        df = df.drop([col], axis=1)
    return df

# average df rows, remove expanded columns and set a new column with a metric name
def count_timeseries(df, columns_to_delete, metric_name, metric_col_name='metric'):
    df['value'] = df.count(axis=1)
    df[metric_col_name] = metric_name
    for col in columns_to_delete:
        df = df.drop([col], axis=1)
    return df

def merge(df, series):
    return pd.merge_asof(df, series, left_index=True, right_index=True, tolerance=pd.Timedelta('1 second')).bfill()    

    

In [291]:
orders_ltcy_df = read_df(file_dir, orders_ltcy_file)
orders_ltcy_df = expand(orders_ltcy_df, base=37)
orders_ltcy_df.head(10) 

,orders_ltcy
date,
2020-03-26 19:47:07,0.24
2020-03-26 19:47:22,0.24
2020-03-26 19:47:37,0.24
2020-03-26 19:47:52,0.24
2020-03-26 19:48:07,0.24
2020-03-26 19:48:22,0.40
2020-03-26 19:48:37,0.39
2020-03-26 19:48:52,0.38
2020-03-26 19:49:07,0.24


In [292]:
carts_ltcy_df = read_df(file_dir, carts_ltcy_file)
carts_ltcy_df = expand(carts_ltcy_df, base=37)
carts_ltcy_df.head() 

,carts_ltcy
date,
2020-03-26 19:46:52,1.96
2020-03-26 19:47:07,0.98
2020-03-26 19:47:22,0.92
2020-03-26 19:47:37,0.49
2020-03-26 19:47:52,0.48


In [293]:
print('orders latency shape {} and carts latency shape {}'
       .format(orders_ltcy_df.shape,carts_ltcy_df.shape))


orders latency shape (6764, 1) and carts latency shape (6765, 1)


In [294]:
services_pods_df = read_df(file_dir, services_pods_file)
services_pods_df = expand(services_pods_df, base=37)
services_pods_df.head(10)  

,carts-db_pods,carts_pods,catalogue-db_pods,catalogue_pods,front-end_pods,orders-db_pods,orders_pods,payment_pods,queue-master_pods,rabbitmq_pods,session-db_pods,shipping_pods,user-db_pods,user_pods
date,,,,,,,,,,,,,,
2020-03-26 19:46:37,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.50,1.00,1.00,1.00,1.00
2020-03-26 19:46:52,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.50,1.00,1.00,1.00,1.00
2020-03-26 19:47:07,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.50,1.00,1.00,1.00,1.00
2020-03-26 19:47:22,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.50,1.00,1.00,1.00,1.00
2020-03-26 19:47:37,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.50,1.00,1.00,1.00,1.00
2020-03-26 19:47:52,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.50,1.00,1.00,1.00,1.00
2020-03-26 19:48:07,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.50,1.00,1.00,1.00,1.00
2020-03-26 19:48:22,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.50,1.00,1.00,1.00,1.00
2020-03-26 19:48:37,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.50,1.00,1.00,1.00,1.00


In [295]:
services_pods_df.shape


(6766, 14)

In [296]:
services_cpu_use_df = read_df(file_dir, service_cpu_usage_file)
services_cpu_use_df = expand(services_cpu_use_df, base=37)
services_cpu_use_df.head(10)    

,carts-db_cpu_use,carts_cpu_use,catalogue-db_cpu_use,catalogue_cpu_use,front-end_cpu_use,orders-db_cpu_use,orders_cpu_use,payment_cpu_use,queue-master_cpu_use,rabbitmq_cpu_use,session-db_cpu_use,shipping_cpu_use,user-db_cpu_use,user_cpu_use
date,,,,,,,,,,,,,,
2020-03-26 19:46:22,0.07,0.03,0.01,0.01,0.03,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
2020-03-26 19:46:37,0.18,0.18,0.01,0.02,0.06,0.02,0.02,0.01,0.02,0.02,0.02,0.01,0.02,0.02
2020-03-26 19:46:52,0.20,0.44,0.01,0.02,0.07,0.02,0.03,0.01,0.02,0.02,0.03,0.01,0.02,0.03
2020-03-26 19:47:07,0.17,0.23,0.01,0.02,0.07,0.02,0.03,0.01,0.02,0.02,0.03,0.01,0.02,0.02
2020-03-26 19:47:22,0.14,0.24,0.01,0.02,0.07,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.02,0.02
2020-03-26 19:47:37,0.18,0.18,0.01,0.02,0.07,0.01,0.03,0.01,0.01,0.01,0.01,0.01,0.02,0.02
2020-03-26 19:47:52,0.18,0.18,0.01,0.01,0.07,0.01,0.03,0.01,0.01,0.01,0.01,0.01,0.02,0.02
2020-03-26 19:48:07,0.17,0.13,0.01,0.01,0.06,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.02,0.02
2020-03-26 19:48:22,0.16,0.12,0.01,0.01,0.07,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.02,0.02


In [297]:
services_cpu_use_df.shape

(6766, 14)

In [298]:
nodes_cpu_use_df = read_df(file_dir, nodes_cpu_usage_file)
nodes_cpu_use_df = expand(nodes_cpu_use_df, base=37)
nodes_cpu_use_df.head()   

,nodes_cpu_use
date,
2020-03-26 19:46:22,2.50
2020-03-26 19:46:37,1.89
2020-03-26 19:46:52,1.35
2020-03-26 19:47:07,1.26
2020-03-26 19:47:22,1.28


In [299]:
orders_req_rate_df = read_df(file_dir, orders_req_rate_file)
orders_req_rate_df = expand(orders_req_rate_df, base=37)
#orders_req_rate_df = orders_req_rate_df[orders_req_rate_df.orders_req  > 0]
orders_req_rate_df.head()

,orders_req
date,
2020-03-26 19:46:52,0.15
2020-03-26 19:47:07,0.13
2020-03-26 19:47:22,0.36
2020-03-26 19:47:37,0.44
2020-03-26 19:47:52,0.44


In [300]:
carts_req_rate_df = read_df(file_dir, carts_req_rate_file)
carts_req_rate_df = expand(carts_req_rate_df, base=37)
#carts_req_rate_df = carts_req_rate_df[carts_req_rate_df.carts_req  > 0]
carts_req_rate_df.head()

,carts_req
date,
2020-03-26 19:46:37,0.47
2020-03-26 19:46:52,0.49
2020-03-26 19:47:07,1.04
2020-03-26 19:47:22,1.62
2020-03-26 19:47:37,1.78


# Orders Flow 

The features selected for the orders workflow are the requests rate, pod-level CPU utilization of the microservices including front-end, orders, users, shipping, payment, cart, users-db, orders- db, cart-db, and the CPU utilization the nodes that host these microservices

In [301]:
selected_services_cpu = ['front-end_cpu_use'
                     ,'orders_cpu_use'  
                     ,'orders-db_cpu_use'
                     ,'user_cpu_use'
                     ,'user-db_cpu_use'
                     ,'shipping_cpu_use'
                     ,'payment_cpu_use'
                     ,'carts_cpu_use'                                                                                
                     ,'carts-db_cpu_use'
                     ]
orders_flow_df = services_cpu_use_df[selected_services_cpu]

# add services pods
selected_services_pods = ['front-end_pods'
                     ,'orders_pods'  
                     #,'orders-db_pods'
                     ,'user_pods'
                     #,'user-db_pods'
                     ,'shipping_pods'
                     ,'payment_pods'
                     ,'carts_pods'                                                                                
                     #,'carts-db_pods'
                     ]
orders_pods_df = services_pods_df[selected_services_pods]
orders_flow_df = merge(orders_flow_df, orders_pods_df)

# add the nodes cpu use
orders_flow_df = merge(orders_flow_df, nodes_cpu_use_df)

# add the orders request rate
orders_flow_df = merge(orders_flow_df, orders_req_rate_df)

# add the orders latency target
orders_flow_df = merge(orders_flow_df, orders_ltcy_df)

orders_flow_df.head(5)



,front-end_cpu_use,orders_cpu_use,orders-db_cpu_use,user_cpu_use,user-db_cpu_use,shipping_cpu_use,payment_cpu_use,carts_cpu_use,carts-db_cpu_use,front-end_pods,orders_pods,user_pods,shipping_pods,payment_pods,carts_pods,nodes_cpu_use,orders_req,orders_ltcy
date,,,,,,,,,,,,,,,,,,
2020-03-26 19:46:22,0.03,0.01,0.01,0.01,0.01,0.01,0.01,0.03,0.07,1.00,1.00,1.00,1.00,1.00,1.00,2.50,0.15,0.24
2020-03-26 19:46:37,0.06,0.02,0.02,0.02,0.02,0.01,0.01,0.18,0.18,1.00,1.00,1.00,1.00,1.00,1.00,1.89,0.15,0.24
2020-03-26 19:46:52,0.07,0.03,0.02,0.03,0.02,0.01,0.01,0.44,0.20,1.00,1.00,1.00,1.00,1.00,1.00,1.35,0.15,0.24
2020-03-26 19:47:07,0.07,0.03,0.02,0.02,0.02,0.01,0.01,0.23,0.17,1.00,1.00,1.00,1.00,1.00,1.00,1.26,0.13,0.24
2020-03-26 19:47:22,0.07,0.02,0.01,0.02,0.02,0.01,0.01,0.24,0.14,1.00,1.00,1.00,1.00,1.00,1.00,1.28,0.36,0.24


In [302]:
orders_flow_df.describe()

,front-end_cpu_use,orders_cpu_use,orders-db_cpu_use,user_cpu_use,user-db_cpu_use,shipping_cpu_use,payment_cpu_use,carts_cpu_use,carts-db_cpu_use,front-end_pods,orders_pods,user_pods,shipping_pods,payment_pods,carts_pods,nodes_cpu_use,orders_req,orders_ltcy
count,"6,766.00","6,766.00","6,766.00","6,766.00","6,766.00","6,766.00","6,766.00","6,766.00","6,766.00","6,766.00","6,766.00","6,766.00","6,766.00","6,766.00","6,766.00","6,766.00","6,766.00","6,766.00"
mean,0.83,0.21,0.02,0.20,0.12,0.05,0.03,0.53,1.83,2.76,2.76,2.76,2.76,2.76,2.76,4.10,5.03,2.97
std,0.62,0.13,0.00,0.13,0.08,0.04,0.01,0.58,1.99,1.10,1.09,1.09,1.09,1.09,1.09,2.08,4.20,3.51
min,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.03,0.01,0.50,1.00,1.00,1.00,1.00,1.00,1.10,0.00,0.02
25%,0.19,0.08,0.02,0.05,0.03,0.04,0.02,0.12,0.02,2.00,2.00,2.00,2.00,2.00,2.00,1.92,0.49,0.24
50%,0.84,0.22,0.02,0.20,0.12,0.05,0.03,0.32,1.15,3.00,3.00,3.00,3.00,3.00,3.00,4.21,5.14,0.49
75%,1.29,0.30,0.02,0.30,0.18,0.06,0.04,0.58,3.28,4.00,4.00,4.00,4.00,4.00,4.00,5.78,8.20,5.81
max,2.73,0.91,0.04,0.64,0.34,0.63,0.10,2.53,6.58,4.00,4.00,4.00,4.00,4.00,4.00,9.66,20.13,9.75


In [303]:
orders_flow_df.shape

(6766, 18)

In [304]:
if 1 == 1:
    processed_file_path = processed_dir + 'orders_flow_data.csv'
    orders_flow_df.to_csv(path_or_buf=processed_file_path, index=True)

# Cart Flow

the features selected for the <b>cart workflow</b> are the request rate, the pod-level CPU utilization of the microservices including front-end, orders, cart, cart-db, and the CPU utilization  the nodes that host these microservices.

In [305]:
selected_services = ['front-end_cpu_use'
                     ,'orders_cpu_use'                                                           
                     ,'carts_cpu_use'                                                                                
                     ,'carts-db_cpu_use'
                     ]
carts_flow_df = services_cpu_use_df[selected_services]

# add the nodes cpu use
carts_flow_df = merge(carts_flow_df, nodes_cpu_use_df)

# add the orders request rate
carts_flow_df = merge(carts_flow_df, carts_req_rate_df)

# add the orders latency target
carts_flow_df = merge(carts_flow_df, carts_ltcy_df)

carts_flow_df.head(5)


,front-end_cpu_use,orders_cpu_use,carts_cpu_use,carts-db_cpu_use,nodes_cpu_use,carts_req,carts_ltcy
date,,,,,,,
2020-03-26 19:46:22,0.03,0.01,0.03,0.07,2.50,0.47,1.96
2020-03-26 19:46:37,0.06,0.02,0.18,0.18,1.89,0.47,1.96
2020-03-26 19:46:52,0.07,0.03,0.44,0.20,1.35,0.49,1.96
2020-03-26 19:47:07,0.07,0.03,0.23,0.17,1.26,1.04,0.98
2020-03-26 19:47:22,0.07,0.02,0.24,0.14,1.28,1.62,0.92


In [306]:
carts_flow_df.shape

(6766, 7)

In [307]:
if 1 == 0:
    processed_file_path = processed_dir + 'cart_flow_data.csv'
    carts_flow_df.to_csv(path_or_buf=processed_file_path, index=True)